In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from mlflow import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
sys.path.append(os.path.abspath('..'))
from src.data_prep import *

In [ ]:
fast_test = True

In [ ]:
data_explorer = DataExplorer()

In [ ]:
data_explorer.application_train

In [ ]:
data_explorer.application_test

Application test is used to make the submission for the competition. It contains the same features as application train except the target variable.

In [ ]:
data_explorer.bureau

In [ ]:
data_explorer.bureau_balance

In [ ]:
data_explorer.credit_card_balance

In [ ]:
data_explorer.installments_payments

In [ ]:
data_explorer.POS_CASH_balance

In [ ]:
data_explorer.previous_application

Search if IDs in application test are present in other datasets.

In [ ]:
# Get the IDs from application test
test_ids = set(data_explorer.application_test['SK_ID_CURR'])
train_ids = set(data_explorer.application_train['SK_ID_CURR'])
test_ids.intersection(train_ids)

In [ ]:
for name, dataset in data_explorer.items():
    print(f"{name}: {dataset.shape}")

# Anomalies

## application_train/test["DAYS_EMPLOYED"]

In [ ]:
(data_explorer.application_train["DAYS_EMPLOYED"] / -365).describe()

In [ ]:
data_explorer.application_train["DAYS_EMPLOYED"].plot.hist()

In [ ]:
print(f'Pourcentage de valeur égales à 365243 : {((data_explorer.application_train["DAYS_EMPLOYED"] == 365243).sum()) / len(data_explorer.application_train) * 100} %')

La colonne étant : "Combien de jours avant la demande, la personne a commencé son emploi actuel". Les chiffres affiché avec le ```.describe()``` sont en années pour une meilleure compréhension.

On voit que le minimum est de -1000 ans et la moyenne est le -174 ans. Il y a donc des valeurs incohérentes.

On voit aussi sur l'histogramme qu'il y a un pic très important à 365243 jours (soit 1000 ans). On a 18 % des données qui ont cette valeur.

In [ ]:
# Créer une nouvelle colonne pour indiquer les valeurs anormales
# data_explorer.application_train['DAYS_EMPLOYED_ANOM'] = data_explorer.application_train["DAYS_EMPLOYED"] == 365243
# data_explorer.application_test['DAYS_EMPLOYED_ANOM'] = data_explorer.application_test["DAYS_EMPLOYED"] == 365243
# Remplacer les valeurs anormales par des NaN
data_explorer.application_train["DAYS_EMPLOYED"] = data_explorer.application_train["DAYS_EMPLOYED"].replace(365243, np.nan)
data_explorer.application_test["DAYS_EMPLOYED"] = data_explorer.application_test["DAYS_EMPLOYED"].replace(365243, np.nan)

# Encoding categorical variables

## application_train and application_test

In [ ]:
cat_cols = data_explorer.application_train.select_dtypes(include=['object']).columns
print(cat_cols)
data_explorer.application_train[cat_cols].describe().T

In [ ]:
# Drop categories 'XNA' which is unknown and will not bring useful information
data_explorer.application_train = data_explorer.application_train[data_explorer.application_train["CODE_GENDER"] != 'XNA']
data_explorer.application_test = data_explorer.application_test[data_explorer.application_test["CODE_GENDER"] != 'XNA']

In [ ]:
nominal = ["CODE_GENDER", "NAME_HOUSING_TYPE"]
ordinal = ["EMERGENCYSTATE_MODE", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "NAME_CONTRACT_TYPE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS"]
not_useful = ["WEEKDAY_APPR_PROCESS_START", "ORGANIZATION_TYPE", "FONDKAPREMONT_MODE", "HOUSETYPE_MODE", "WALLSMATERIAL_MODE", "OCCUPATION_TYPE"]

In [ ]:
data_explorer.application_train, data_explorer.application_test = encode_df(data_explorer.application_train, nominal=nominal, ordinal=ordinal, df_test=data_explorer.application_test)
for col in not_useful:
    data_explorer.application_train.drop(columns=[col], inplace=True)
    data_explorer.application_test.drop(columns=[col], inplace=True)

## Bureau

In [ ]:
cat_cols = data_explorer.bureau.select_dtypes(include=['object']).columns
print(cat_cols)
data_explorer.bureau[cat_cols].describe().T

In [ ]:
data_explorer.bureau["CREDIT_TYPE"].value_counts()

In [ ]:
# group_rare_categories(data_explorer.bureau)

In [ ]:
nominal = ["CREDIT_CURRENCY", "CREDIT_TYPE"]
ordinal = ["CREDIT_ACTIVE"]
data_explorer.bureau, _ = encode_df(data_explorer.bureau, nominal=nominal, ordinal=ordinal)

## Bureau balance

In [ ]:
cat_cols = data_explorer.bureau_balance.select_dtypes(include=['object']).columns
print(cat_cols)
data_explorer.bureau_balance[cat_cols].describe().T

In [ ]:
data_explorer.bureau_balance["STATUS"].value_counts()

In [ ]:
# group_rare_categories(data_explorer.bureau_balance)

In [ ]:
ordinal = ["STATUS"]
data_explorer.bureau_balance, _ = encode_df(data_explorer.bureau_balance, ordinal=ordinal)

## Credit card balance

In [ ]:
cat_cols = data_explorer.credit_card_balance.select_dtypes(include=['object']).columns
print(cat_cols)
data_explorer.credit_card_balance[cat_cols].describe().T

In [ ]:
data_explorer.credit_card_balance["NAME_CONTRACT_STATUS"].value_counts()

In [ ]:
# group_rare_categories(data_explorer.credit_card_balance)

In [ ]:
ordinal = ["NAME_CONTRACT_STATUS"]
data_explorer.credit_card_balance, _ = encode_df(data_explorer.credit_card_balance, ordinal=ordinal)

## Installments payments

In [ ]:
cat_cols = data_explorer.installments_payments.select_dtypes(include=['object']).columns
print(cat_cols)

## POS CASH balance

In [ ]:
cat_cols = data_explorer.POS_CASH_balance.select_dtypes(include=['object']).columns
print(cat_cols)
data_explorer.POS_CASH_balance[cat_cols].describe().T

In [ ]:
data_explorer.POS_CASH_balance["NAME_CONTRACT_STATUS"].value_counts()

In [ ]:
# group_rare_categories(data_explorer.POS_CASH_balance)

In [ ]:
ordinal = ["NAME_CONTRACT_STATUS"]
data_explorer.POS_CASH_balance, _ = encode_df(data_explorer.POS_CASH_balance, ordinal=ordinal)

## Previous application

In [ ]:
cat_cols = data_explorer.previous_application.select_dtypes(include=['object']).columns
print(cat_cols)
data_explorer.previous_application[cat_cols].describe().T

In [ ]:
for col in cat_cols:
    print(f"{data_explorer.previous_application[col].value_counts()}\n")

In [ ]:
# Colonnes à simplifier : celles avec une forte cardinalité ou beaucoup de 'XNA'/'XAP' rares
# On utilise un seuil de 1% (0.01)
# cols_to_simplify = [
#     "NAME_CASH_LOAN_PURPOSE",
#     "NAME_GOODS_CATEGORY",
#     "CODE_REJECT_REASON",
#     "NAME_TYPE_SUITE",
#     "NAME_SELLER_INDUSTRY",
#     "PRODUCT_COMBINATION"
# ]

# group_rare_categories(data_explorer.previous_application)

In [ ]:
nominal = ["NAME_CONTRACT_TYPE", "WEEKDAY_APPR_PROCESS_START",
       "NAME_CONTRACT_STATUS", "NAME_PAYMENT_TYPE", "CODE_REJECT_REASON",
       "NAME_TYPE_SUITE", "NAME_CLIENT_TYPE", "NAME_PORTFOLIO",
       "NAME_PRODUCT_TYPE", "CHANNEL_TYPE", "NAME_SELLER_INDUSTRY"]
ordinal = ["NAME_YIELD_GROUP"]
not_useful = ["FLAG_LAST_APPL_PER_CONTRACT", "NAME_CASH_LOAN_PURPOSE",  "NAME_GOODS_CATEGORY", "PRODUCT_COMBINATION"]

In [ ]:
data_explorer.previous_application, _ = encode_df(data_explorer.previous_application, nominal=nominal, ordinal=ordinal)

## Work with missing values

In [ ]:
if not fast_test:
    for name, dataset in data_explorer.items():
        null_value_chart(dataset, name, subdir="preprocessing")

In [ ]:
# Print percentage of missing values of columns for each column in all datasets
threshold = 60  # percentage threshold
for name, dataset in data_explorer.items():
    print(f"{name}:")
    for col in dataset.columns:
        missing_percentage = dataset[col].isna().mean() * 100
        if missing_percentage > threshold:
            print(f"  {col}: {missing_percentage:.2f}% missing")

Delete columns with more than 60% null values because they can't bring much information.

In [ ]:
for name, dataset in data_explorer.items():
    print(f"{name}: ", end="")
    data_explorer[name] = drop_cols_above_threshold(dataset, threshold=0.65, verbose=1)

In [ ]:
if not fast_test:
    for name, dataset in data_explorer.items():
        null_value_chart(dataset, name, subdir="postprocessing")

# Join tables

1. ``bureau_balance``: join with ``bureau`` on SK_ID_BUREAU
2. ``bureau``: join with ``application_train`` and ``application_test`` on SK_ID_CURR
3. ``POS_CASH_balance``: join with ``previous_application`` on SK_ID_PREV
4. ``installments_payments``: join with ``previous_application`` on SK_ID_PREV
5. ``credit_card_balance``: join with ``previous_application `` on SK_ID_PREV
6. ``previous_application``: join with ``application_train`` and ``application_test`` on SK_ID_CURR
7. ``installments_payments``: join with ``application_train`` and ``application_test`` on SK_ID_CURR
8. ``credit_card_balance``: join with ``application_train`` and ``application_test`` on SK_ID_CURR
9. ``POS_CASH_balance``: join with ``application_train`` and ``application_test`` on SK_ID_CURR
10. Save enriched ``application_train`` and ``application_test``

## 1. Merge ``bureau`` and ``bureau_balance``

In [ ]:
data_explorer.bureau

In [ ]:
data_explorer.bureau_balance

### 1.1. Transformer la colonne STATUS en numérique

In [ ]:
status_mapping = {
    'C': 0,   # Crédit clos
    'X': 0,   # Statut inconnu
    '0': 0,   # Pas de retard
    '1': 1,   # Retard 1-30 jours
    '2': 2,   # Retard 31-60 jours
    '3': 3,   # Retard 61-90 jours
    '4': 4,   # Retard 91-120 jours
    '5': 5    # Retard 120+ ou radié
}

In [ ]:
data_explorer.bureau_balance['STATUS_NUM'] = data_explorer.bureau_balance['STATUS'].map(status_mapping)

### 1.2. Agréger `bureau_balance` par crédit (SK_ID_BUREAU)

In [ ]:
bb_agg = data_explorer.bureau_balance.groupby('SK_ID_BUREAU').agg(
    MONTHS_BALANCE_MIN=('MONTHS_BALANCE', 'min'),
    MONTHS_BALANCE_MAX=('MONTHS_BALANCE', 'max'),
    MAX_DPD=('STATUS_NUM', 'max'),                 # Retard maximal sur tout le crédit
    MEAN_DPD=('STATUS_NUM', 'mean'),              # Retard moyen
    ON_TIME_RATIO=('STATUS_NUM', lambda x: (x==0).sum() / len(x)),  # % des mois payés à temps
    LATE_MONTHS=('STATUS_NUM', lambda x: (x>0).sum())               # Nombre de mois en retard
).reset_index() #


In [ ]:
bb_agg

### 1.3. Joindre `bureau_balance` agrégé à `bureau`

In [ ]:
bureau = data_explorer.bureau.merge(bb_agg, on='SK_ID_BUREAU', how='left')

In [ ]:
bureau

In [ ]:
for col in bureau.columns:
    missing_percentage = bureau[col].isna().mean() * 100
    if missing_percentage > threshold:
        print(f"  {col}: {missing_percentage:.2f}% missing")

In [ ]:
null_value_chart(bureau, save=False)

### 1.4. Agréger `bureau` par client (SK_ID_CURR)

In [ ]:
bureau_client_agg = bureau.groupby('SK_ID_CURR').agg(
    NB_CREDITS_TOTAL=('SK_ID_BUREAU', 'count'),
    NB_CREDITS_ACTIVE=('CREDIT_ACTIVE', lambda x: (x=='Active').sum()),
    DAYS_CREDIT_MIN=('DAYS_CREDIT', 'min'),
    DAYS_CREDIT_MAX=('DAYS_CREDIT', 'max'),
    DAYS_CREDIT_MEAN=('DAYS_CREDIT', 'mean'),
    CREDIT_SUM_TOTAL=('AMT_CREDIT_SUM', 'sum'),
    CREDIT_SUM_MEAN=('AMT_CREDIT_SUM', 'mean'),
    MAX_DPD_CLIENT=('MAX_DPD', 'max'),
    MEAN_DPD_CLIENT=('MEAN_DPD', 'mean'),
    ON_TIME_RATIO_CLIENT=('ON_TIME_RATIO', 'mean'),
    LATE_MONTHS_CLIENT=('LATE_MONTHS', 'sum')
).reset_index()

In [ ]:
bureau_client_agg

## 2. Merge ``bureau`` with ``application_test`` and ``application_train``

In [ ]:
application_train = data_explorer.application_train.merge(bureau_client_agg, on='SK_ID_CURR', how='left')
application_test = data_explorer.application_test.merge(bureau_client_agg, on='SK_ID_CURR', how='left')

In [ ]:
application_train

## 3. Merge ``POS_CASH_balance`` with ``previous_application``

Voir s'il y des chaines d'IDs

In [ ]:
prev_ids = set(data_explorer.POS_CASH_balance.SK_ID_PREV)
curr_ids = set(data_explorer.POS_CASH_balance.SK_ID_CURR)
prev_ids.intersection(curr_ids)

In [ ]:
pos_prev_agg = data_explorer.POS_CASH_balance.groupby('SK_ID_PREV').agg({
    'CNT_INSTALMENT': ['sum','mean'],
    'SK_DPD': ['max','mean'],
    'SK_DPD_DEF': ['max','mean']
})

pos_prev_agg.columns = ['_'.join(col).strip() for col in pos_prev_agg.columns.values]
pos_prev_agg.reset_index(inplace=True)

# Joindre à previous_application
prev = data_explorer.previous_application.merge(pos_prev_agg, on='SK_ID_PREV', how='left')

## 4. Merge ``installments_payments`` with ``previous_application``

In [ ]:
inst_prev_agg = data_explorer.installments_payments.groupby('SK_ID_PREV').agg({
    'AMT_INSTALMENT': ['sum','mean'],
    'AMT_PAYMENT': ['sum','mean'],
    'DAYS_INSTALMENT': ['min','max','mean'],
    'DAYS_ENTRY_PAYMENT': ['min','max','mean'],
})

inst_prev_agg.columns = ['_'.join(col).strip() for col in inst_prev_agg.columns.values]
inst_prev_agg.reset_index(inplace=True)

# Joindre à previous_application
prev = prev.merge(inst_prev_agg, on='SK_ID_PREV', how='left')

## 5. ``credit_card_balance``: join with ``previous_application `` on SK_ID_PREV

In [ ]:
#card_prev_agg = data_explorer.credit_card_balance.groupby('SK_ID_PREV').agg({
#    'CNT_INSTALMENT': ['sum','mean'],
#    'SK_DPD': ['max','mean'],
#    'SK_DPD_DEF': ['max','mean']
#})
#
#card_prev_agg.columns = ['_'.join(col).strip() for col in card_prev_agg.columns.values]
#card_prev_agg.reset_index(inplace=True)
#
## Joindre à previous_application
#prev = prev.merge(card_prev_agg, on='SK_ID_PREV', how='left')

In [ ]:
prev

In [ ]:
null_cols_above_threshold(prev, 0.6)

In [ ]:
null_value_chart(prev, save=False)

## 6. Merge ``previous_application`` with ``application_train`` and ``application_test``

In [ ]:
prev

In [ ]:
# Sélectionner seulement les colonnes numériques sauf SK_ID_CURR
numeric_cols = prev.select_dtypes(exclude=['object']).columns
numeric_cols = [col for col in numeric_cols if not col in ['SK_ID_CURR', 'SK_ID_PREV']]

# Agrégation par client
prev_client_agg = prev.groupby('SK_ID_CURR')[numeric_cols].agg(['sum','mean','max','min']).reset_index()
prev_client_agg.columns = [
    f"{col[0]}_{col[1]}" if col[1] != '' else col[0]
    for col in prev_client_agg.columns
]

In [ ]:
# Fusion avec application
application_train = application_train.merge(prev_client_agg, on='SK_ID_CURR', how='left')
application_test = application_test.merge(prev_client_agg, on='SK_ID_CURR', how='left')

In [ ]:
null_cols_above_threshold(application_train, threshold/100)

## 7. ``installments_payments``: join with ``application_train`` and ``application_test`` on SK_ID_CURR

In [ ]:
inst_app_agg = data_explorer.installments_payments.groupby('SK_ID_CURR').agg({
    'AMT_INSTALMENT': ['sum','mean'],
    'AMT_PAYMENT': ['sum','mean'],
    'DAYS_INSTALMENT': ['min','max','mean'],
    'DAYS_ENTRY_PAYMENT': ['min','max','mean'],
})

inst_app_agg.columns = ['_'.join(col).strip() for col in inst_app_agg.columns.values]
inst_app_agg.reset_index(inplace=True)

# Joindre à application
application_train = application_train.merge(inst_app_agg, on='SK_ID_CURR', how='left')
application_test = application_test.merge(inst_app_agg, on='SK_ID_CURR', how='left')

## 8.``credit_card_balance``: join with ``application_train`` and ``application_test`` on SK_ID_CURR

In [ ]:
data_explorer.credit_card_balance.info()

In [ ]:
numeric_cols = data_explorer.credit_card_balance.select_dtypes(exclude=['object']).columns
numeric_cols = [col for col in numeric_cols if col != 'SK_ID_CURR']

cred_app_agg = data_explorer.credit_card_balance.groupby('SK_ID_CURR')[numeric_cols].agg(['sum','mean','max','min'])

cred_app_agg.columns = ['_'.join(col).strip() for col in cred_app_agg.columns.values]
cred_app_agg.reset_index(inplace=True)

# Joindre à application
application_train = application_train.merge(cred_app_agg, on='SK_ID_CURR', how='left')
application_test = application_test.merge(cred_app_agg, on='SK_ID_CURR', how='left')

## 9. ``POS_CASH_balance``: join with ``application_train`` and ``application_test`` on SK_ID_CURR

In [ ]:
data_explorer.POS_CASH_balance.info()

In [ ]:
pos_app_agg = data_explorer.POS_CASH_balance.groupby('SK_ID_CURR').agg({
    'CNT_INSTALMENT': ['sum','mean'],
    'SK_DPD': ['max','mean'],
    'SK_DPD_DEF': ['max','mean']
})

pos_app_agg.columns = ['_'.join(col).strip() for col in pos_app_agg.columns.values]
pos_app_agg.reset_index(inplace=True)

# Joindre à application
application_train = application_train.merge(pos_app_agg, on='SK_ID_CURR', how='left')
application_test = application_test.merge(pos_app_agg, on='SK_ID_CURR', how='left')

## 10. Save enriched ``application_train`` and ``application_test``

In [ ]:
null_cols_above_threshold(application_train)

In [ ]:
null_value_chart(application_train, save=False)

In [ ]:
application_train.to_csv("../model/application_train_enriched.csv", index=False)
application_test.to_csv("../model/application_test_enriched.csv", index=False)